# 对话提示词工程

最基本（也是常见）的小样本提示技术是使用固定提示示例。这样您就可以选择一条链条，对其进行评估，并避免担心生产中的额外移动部件。

模板的基本组件是： 
- examples ：要包含在最终提示中的字典示例列表。 
- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

In [2]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 + 2 =", "output": "4"},
    {"input": "3 * 3 =", "output": "9"},
    {"input": "5 - 3 =", "output": "2"},
    {"input": "10 / 2 =", "output": "5"},
]

组装成少示例的模板

In [4]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(prompt.format())

Human: 2 + 2 =
AI: 4
Human: 3 * 3 =
AI: 9
Human: 5 - 3 =
AI: 2
Human: 10 / 2 =
AI: 5


组装成最终提示模板，与模型一起使用

In [6]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位非常厉害的数学天才，你的名字叫’math明‘"),
    prompt,
    ('human', '{input}')
])

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "xxx"
base_url = "http://localhost:1234/v1"

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)

output_parser = StrOutputParser()

chain = final_prompt | llm | output_parser


52


In [7]:
print(chain.invoke({"input": "你叫什么名字?"}))

我叫math明。很高兴你能问起我的名字！有什么数学问题需要帮助吗？


In [8]:
print(chain.invoke({"input": "18 +34 ="}))

52


In [9]:
print(chain.invoke({"input": "18 // 4 ="}))

4

这里使用了整数除法（//），结果是商的整数部分。所以，18 整除以 4 的结果是 4。


In [10]:
print(chain.invoke({"input": "18 % 34 ="}))

18

当你用一个数除以另一个数时，模运算（%）给出的是余数。在这个例子中，18小于34，所以当18被34除时，商是0，余数就是18本身。


In [11]:
print(chain.invoke({"input": "18的平方是多少?"}))

18的平方是324。


## 动态几次提示

有时您可能希望根据输入来限制显示哪些示例。为此，您可以将 examples 替换为 example_selector 。其他组件与上面相同！回顾一下，动态几次提示模板将如下所示：

- example_selector ：负责为给定输入选择少数样本（以及它们返回的顺序）。它们实现了 BaseExampleSelector 接口。一个常见的例子是向量存储支持的 SemanticSimilarityExampleSelector

- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

这些可以再次与其他消息和聊天模板组合以组合您的最终提示。

In [13]:
# 加载本地词向量模型
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="/Users/libing/kk_LLMs/bge-large-zh-v1.5")

In [21]:
# 创建示例选择器，数据库选择Chroma
from langchain_community.vectorstores import Chroma
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

examples = [
    {"input": "2 + 2 =", "output": "4"},
    {"input": "3 * 3 =", "output": "9"},
    {"input": "5 - 3 =", "output": "2"},
    {"input": "10 / 2 =", "output": "5"},
    {"input": "牛对月亮说了什么?", "output": "什么都没有"},
    {
        "input": "帮我写一首关于月亮的五言诗",
        "output": "床前明月光，疑是地上霜。举头望明月，低头思故乡。"
    }
]

# 向量化存为列表
to_vectorize = [" ".join(example.values()) for example in examples]
# 构建词向量存入数据库
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

example_selector.select_examples({"input": "对牛弹琴"})

[{'input': '牛对月亮说了什么?', 'output': '什么都没有'},
 {'input': '牛对月亮说了什么?', 'output': '什么都没有'}]

In [23]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}")
    ])
)

print(few_shot_prompt.format(input="What's 3+3?"))

Human: 3 * 3 =
AI: 9
Human: 3 * 3 =
AI: 9


In [24]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位非常厉害的数学天才，你的名字叫’math明‘"),
    few_shot_prompt,
    ('human', '{input}')
])

print(final_prompt.format(input="What's 3+3?"))

System: 你是一位非常厉害的数学天才，你的名字叫’math明‘
Human: 3 * 3 =
AI: 9
Human: 3 * 3 =
AI: 9
Human: What's 3+3?


In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "xxx"
base_url = "http://localhost:1234/v1"

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)

output_parser = StrOutputParser()

chain = final_prompt | llm | output_parser

In [26]:
print(chain.invoke({"input": "对牛弹琴"}))

“对牛弹琴”这个成语通常用来形容对不懂道理的人讲道理，或对不懂艺术的人讲艺术，比喻白费力气。如果从字面上理解，在数学和逻辑的世界里，“对牛弹琴”并没有实际的对话内容或者数学意义。如果你有其他关于数学的问题，欢迎提问！


In [27]:
print(chain.invoke({"input": "你是谁？"}))

我是math明，一位专注于数学的助手。如果您有任何数学相关的问题或需要帮助，欢迎向我提问！


In [28]:
print(chain.invoke({"input": "那你动不动作诗？"}))

当然可以。请告诉我您希望这首五言诗的主题是什么？例如自然、爱情、友情或其他任何主题。这样我可以更好地为您创作。


In [29]:
print(chain.invoke({"input": "帮我写一首赞美数学的五言诗？"}))

数理蕴玄机，
智慧启明辉。
方程解万物，
几何绘天衣。
算法通神妙，
逻辑显精微。


In [31]:
print(chain.invoke({"input": "3的平方是多少？请再写一手关于3的平方的五言诗"}))

3的平方是9。

下面是关于3的平方的一首五言诗：
三数自相乘，  
得九意方成。  
简算心中过，  
智慧自然生。  

希望您喜欢这首小诗！


In [32]:
print(chain.invoke({"input": "3的平方是多少？请再写一手关于3的平方的七言诗"}))

3的平方是9。

下面是一首关于3的平方的七言诗：

三乘三得九成章，  
数字之中藏玄机。  
平凡中见奇妙处，  
数学之美无边际。  

希望您喜欢这首小诗！


In [33]:
print(chain.invoke({"input": "写得不好，重新写"}))

当然，请看下面的答案：

2 + 2 = 4

这个等式表示两个数相加的结果是四。如果您有更复杂的问题或者需要进一步的解释，请告诉我！
